# Βελτιστοποίηση Μοντέλων Open AI

Αυτό το σημειωματάριο βασίζεται στις τρέχουσες οδηγίες που παρέχονται στην τεκμηρίωση [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) από την Open AI.

Η βελτιστοποίηση βελτιώνει την απόδοση των θεμελιωδών μοντέλων για την εφαρμογή σας, εκπαιδεύοντάς τα εκ νέου με επιπλέον δεδομένα και πλαίσιο που σχετίζονται με τη συγκεκριμένη χρήση ή σενάριο. Σημειώστε ότι οι τεχνικές μηχανικής προτροπής όπως η _few shot learning_ και η _retrieval augmented generation_ σας επιτρέπουν να ενισχύσετε την προεπιλεγμένη προτροπή με σχετικά δεδομένα για να βελτιώσετε την ποιότητα. Ωστόσο, αυτές οι προσεγγίσεις περιορίζονται από το μέγιστο μέγεθος παραθύρου token του στοχευόμενου θεμελιώδους μοντέλου.

Με τη βελτιστοποίηση, επανεκπαιδεύουμε ουσιαστικά το ίδιο το μοντέλο με τα απαιτούμενα δεδομένα (επιτρέποντάς μας να χρησιμοποιήσουμε πολλά περισσότερα παραδείγματα από όσα χωρούν στο μέγιστο παράθυρο token) - και αναπτύσσουμε μια _προσαρμοσμένη_ έκδοση του μοντέλου που δεν χρειάζεται πλέον να παρέχονται παραδείγματα κατά τον χρόνο συμπερασμού. Αυτό όχι μόνο βελτιώνει την αποτελεσματικότητα του σχεδιασμού της προτροπής μας (έχουμε μεγαλύτερη ευελιξία στη χρήση του παραθύρου token για άλλα πράγματα) αλλά ενδεχομένως βελτιώνει και το κόστος μας (μειώνοντας τον αριθμό των tokens που πρέπει να στείλουμε στο μοντέλο κατά τον χρόνο συμπερασμού).

Η βελτιστοποίηση έχει 4 βήματα:
1. Προετοιμάστε τα δεδομένα εκπαίδευσης και ανεβάστε τα.
1. Εκτελέστε τη δουλειά εκπαίδευσης για να αποκτήσετε ένα βελτιστοποιημένο μοντέλο.
1. Αξιολογήστε το βελτιστοποιημένο μοντέλο και επαναλάβετε για ποιότητα.
1. Αναπτύξτε το βελτιστοποιημένο μοντέλο για συμπερασμό όταν είστε ικανοποιημένοι.

Σημειώστε ότι δεν υποστηρίζουν όλα τα θεμελιώδη μοντέλα τη βελτιστοποίηση - [ελέγξτε την τεκμηρίωση OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) για τις πιο πρόσφατες πληροφορίες. Μπορείτε επίσης να βελτιστοποιήσετε ένα μοντέλο που έχει ήδη βελτιστοποιηθεί προηγουμένως. Σε αυτό το σεμινάριο, θα χρησιμοποιήσουμε το `gpt-35-turbo` ως το στοχευόμενο θεμελιώδες μοντέλο για βελτιστοποίηση.

---


### Βήμα 1.1: Προετοιμάστε το Σύνολο Δεδομένων σας

Ας δημιουργήσουμε ένα chatbot που θα σας βοηθά να κατανοήσετε τον περιοδικό πίνακα των στοιχείων απαντώντας σε ερωτήσεις για ένα στοιχείο με ένα λιμερικό. Σε _αυτό_ το απλό σεμινάριο, θα δημιουργήσουμε απλώς ένα σύνολο δεδομένων για να εκπαιδεύσουμε το μοντέλο με μερικά δείγματα απαντήσεων που δείχνουν τη μορφή που αναμένεται από τα δεδομένα. Σε μια πραγματική περίπτωση χρήσης, θα χρειαστεί να δημιουργήσετε ένα σύνολο δεδομένων με πολλά περισσότερα παραδείγματα. Επίσης, μπορεί να μπορείτε να χρησιμοποιήσετε ένα ανοιχτό σύνολο δεδομένων (για τον τομέα εφαρμογής σας) αν υπάρχει, και να το αναμορφώσετε για χρήση στην εκπαίδευση.

Επειδή εστιάζουμε στο `gpt-35-turbo` και αναζητούμε απάντηση μίας μόνο γύρας (συμπλήρωση συνομιλίας), μπορούμε να δημιουργήσουμε παραδείγματα χρησιμοποιώντας [αυτήν την προτεινόμενη μορφή](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) που αντανακλά τις απαιτήσεις συμπλήρωσης συνομιλίας της OpenAI. Αν περιμένετε περιεχόμενο πολλαπλών γύρων συνομιλίας, θα χρησιμοποιούσατε τη [μορφή παραδείγματος πολλαπλών γύρων](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) που περιλαμβάνει μια παράμετρο `weight` για να υποδείξει ποια μηνύματα πρέπει να χρησιμοποιηθούν (ή όχι) στη διαδικασία εκπαίδευσης.

Θα χρησιμοποιήσουμε τη πιο απλή μορφή μίας μόνο γύρας για το σεμινάριό μας εδώ. Τα δεδομένα είναι σε μορφή [jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) με 1 εγγραφή ανά γραμμή, κάθε μία αναπαριστάται ως αντικείμενο μορφοποιημένο σε JSON. Το απόσπασμα παρακάτω δείχνει 2 εγγραφές ως δείγμα - δείτε το [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) για το πλήρες δείγμα (10 παραδείγματα) που θα χρησιμοποιήσουμε για το σεμινάριο εκπαίδευσης. **Σημείωση:** Κάθε εγγραφή _πρέπει_ να ορίζεται σε μία μόνο γραμμή (όχι διαχωρισμένη σε πολλές γραμμές όπως είναι τυπικό σε ένα μορφοποιημένο αρχείο JSON)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Σε μια πραγματική περίπτωση χρήσης θα χρειαστείτε ένα πολύ μεγαλύτερο σύνολο παραδειγμάτων για καλά αποτελέσματα - η ανταλλαγή θα είναι μεταξύ της ποιότητας των απαντήσεων και του χρόνου/κόστους για την εκπαίδευση. Χρησιμοποιούμε ένα μικρό σύνολο ώστε να ολοκληρώσουμε γρήγορα την εκπαίδευση για να δείξουμε τη διαδικασία. Δείτε [αυτό το παράδειγμα OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) για ένα πιο σύνθετο σεμινάριο εκπαίδευσης.


---

### Βήμα 1.2 Μεταφόρτωση του Συνόλου Δεδομένων σας

Μεταφορτώστε τα δεδομένα χρησιμοποιώντας το Files API [όπως περιγράφεται εδώ](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Σημειώστε ότι για να εκτελέσετε αυτόν τον κώδικα, πρέπει πρώτα να έχετε κάνει τα εξής βήματα:
 - Να έχετε εγκαταστήσει το πακέτο Python `openai` (βεβαιωθείτε ότι χρησιμοποιείτε έκδοση >=0.28.0 για τις πιο πρόσφατες λειτουργίες)
 - Να έχετε ορίσει τη μεταβλητή περιβάλλοντος `OPENAI_API_KEY` με το κλειδί API του OpenAI σας
Για να μάθετε περισσότερα, δείτε τον [Οδηγό Ρύθμισης](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) που παρέχεται για το μάθημα.

Τώρα, εκτελέστε τον κώδικα για να δημιουργήσετε ένα αρχείο για μεταφόρτωση από το τοπικό σας αρχείο JSONL.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Βήμα 2.1: Δημιουργήστε την εργασία Fine-tuning με το SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Βήμα 2.2: Έλεγχος της Κατάστασης της εργασίας

Εδώ είναι μερικά πράγματα που μπορείτε να κάνετε με το API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Λίστα με τις τελευταίες n εργασίες fine-tuning
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Λήψη λεπτομερειών για μια συγκεκριμένη εργασία fine-tuning
- `client.fine_tuning.jobs.cancel(<job_id>)` - Ακύρωση μιας εργασίας fine-tuning
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Λίστα με έως n γεγονότα από την εργασία
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Το πρώτο βήμα της διαδικασίας είναι _η επικύρωση του αρχείου εκπαίδευσης_ για να βεβαιωθείτε ότι τα δεδομένα είναι στη σωστή μορφή.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Βήμα 2.3: Παρακολούθηση γεγονότων για την παρακολούθηση της προόδου


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Βήμα 2.4: Προβολή κατάστασης στον Πίνακα Ελέγχου OpenAI


Μπορείτε επίσης να δείτε την κατάσταση επισκεπτόμενοι την ιστοσελίδα της OpenAI και εξερευνώντας την ενότητα _Fine-tuning_ της πλατφόρμας. Αυτό θα σας δείξει την κατάσταση της τρέχουσας εργασίας, και επίσης θα σας επιτρέψει να παρακολουθήσετε το ιστορικό των προηγούμενων εκτελέσεων εργασιών. Σε αυτήν την εικόνα, μπορείτε να δείτε ότι η προηγούμενη εκτέλεση απέτυχε, και η δεύτερη εκτέλεση πέτυχε. Για το πλαίσιο, αυτό συνέβη όταν η πρώτη εκτέλεση χρησιμοποίησε ένα αρχείο JSON με λανθασμένα μορφοποιημένες εγγραφές - μόλις διορθώθηκε, η δεύτερη εκτέλεση ολοκληρώθηκε με επιτυχία και έκανε το μοντέλο διαθέσιμο για χρήση.

![Fine-tuning job status](../../../../../translated_images/el/fine-tuned-model-status.563271727bf7bfba.webp)


Μπορείτε επίσης να δείτε τα μηνύματα κατάστασης και τις μετρήσεις μετακινούμενοι προς τα κάτω στο οπτικό ταμπλό όπως φαίνεται:

| Μηνύματα | Μετρήσεις |
|:---|:---|
| ![Messages](../../../../../translated_images/el/fine-tuned-messages-panel.4ed0c2da5ea1313b.webp) |  ![Metrics](../../../../../translated_images/el/fine-tuned-metrics-panel.700d7e4995a65229.webp)|


---

### Βήμα 3.1: Ανάκτηση ID & Δοκιμή του Μοντέλου με Fine-Tuning στον Κώδικα


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Βήμα 3.2: Φόρτωση & Δοκιμή του Εξατομικευμένου Μοντέλου στο Playground

Τώρα μπορείτε να δοκιμάσετε το εξατομικευμένο μοντέλο με δύο τρόπους. Πρώτον, μπορείτε να επισκεφτείτε το Playground και να χρησιμοποιήσετε το αναπτυσσόμενο μενού Models για να επιλέξετε το νέο εξατομικευμένο μοντέλο από τις διαθέσιμες επιλογές. Η άλλη επιλογή είναι να χρησιμοποιήσετε την επιλογή "Playground" που εμφανίζεται στον πίνακα Fine-tuning (δείτε το στιγμιότυπο οθόνης παραπάνω), η οποία εκκινεί την ακόλουθη _συγκριτική_ προβολή που δείχνει τις εκδόσεις του βασικού και του εξατομικευμένου μοντέλου δίπλα-δίπλα για γρήγορη αξιολόγηση.

![Fine-tuning job status](../../../../../translated_images/el/fine-tuned-playground-compare.56e06f0ad8922016.webp)

Απλώς συμπληρώστε το πλαίσιο συστήματος που χρησιμοποιήθηκε στα δεδομένα εκπαίδευσής σας και δώστε την ερώτηση δοκιμής σας. Θα παρατηρήσετε ότι και οι δύο πλευρές ενημερώνονται με το ίδιο πλαίσιο και ερώτηση. Εκτελέστε τη σύγκριση και θα δείτε τη διαφορά στις απαντήσεις μεταξύ τους. _Σημειώστε πώς το εξατομικευμένο μοντέλο αποδίδει την απάντηση στη μορφή που παρείχατε στα παραδείγματά σας, ενώ το βασικό μοντέλο απλώς ακολουθεί την εντολή του συστήματος_.

![Fine-tuning job status](../../../../../translated_images/el/fine-tuned-playground-launch.5a26495c983c6350.webp)

Θα παρατηρήσετε επίσης ότι η σύγκριση παρέχει τους μετρητές tokens για κάθε μοντέλο, καθώς και τον χρόνο που απαιτήθηκε για την εκτέλεση της πρόβλεψης. **Αυτό το συγκεκριμένο παράδειγμα είναι απλοϊκό και έχει σκοπό να δείξει τη διαδικασία, αλλά δεν αντικατοπτρίζει πραγματικό σύνολο δεδομένων ή σενάριο**. Μπορεί να παρατηρήσετε ότι και τα δύο δείγματα εμφανίζουν τον ίδιο αριθμό tokens (το πλαίσιο συστήματος και η εντολή χρήστη είναι ταυτόσημα), με το εξατομικευμένο μοντέλο να απαιτεί περισσότερο χρόνο για την εκτέλεση της πρόβλεψης (προσαρμοσμένο μοντέλο).

Σε πραγματικά σενάρια, δεν θα χρησιμοποιείτε ένα απλό παράδειγμα όπως αυτό, αλλά θα κάνετε fine-tuning με πραγματικά δεδομένα (π.χ., κατάλογο προϊόντων για εξυπηρέτηση πελατών), όπου η ποιότητα της απάντησης θα είναι πολύ πιο εμφανής. Σε _αυτό_ το πλαίσιο, η επίτευξη ισοδύναμης ποιότητας απάντησης με το βασικό μοντέλο θα απαιτήσει περισσότερη προσαρμοσμένη μηχανική προτροπών, η οποία θα αυξήσει τη χρήση tokens και πιθανώς τον σχετικό χρόνο επεξεργασίας για την πρόβλεψη. _Για να το δοκιμάσετε, δείτε τα παραδείγματα fine-tuning στο OpenAI Cookbook για να ξεκινήσετε._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Αποποίηση ευθυνών**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης AI [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που επιδιώκουμε την ακρίβεια, παρακαλούμε να λάβετε υπόψη ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν λάθη ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή λανθασμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
